# LAB3 - NERC with CRF

### Credits

The content of this notebook is an adaptation of:
https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/

which is itself based on:

https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html


### Background

We denote the input sequence (the words in a sentence):

$$x = (x_1,\dots, x_m)$$

The sequence of output states, i.e. the named entity tags, is represented as:

$$s = (s_1,\dots, s_m)$$

In conditional random fields we model the conditional probability:

$$p(s_1,\dots,s_m|x_1,\dots,x_m)$$

We do this by define a feature map that maps an entire input sequence x paired with an entire state sequence s to some d-dimensional feature vector:

$$\Phi(x_1,\dots,x_m,s_1,\dots,s_m)\in\mathbb{R}^d$$

Then we can model the probability as a log-linear model with the parameter vector `w`:

$$p(s|x; w) = \frac{\exp(w\cdot\Phi(x, s))}{\sum_{s^\prime} \exp(w\cdot\Phi(x, s^\prime))},$$

Here s' ranges over all possible output sequences. For the estimation of w, we assume that we have a set of n labeled examples. Now we define the regularized log-likelihood function L:



$$L(w) = \sum_{i=1}^n \log p(s^i|x^i; w) - \frac{\lambda_2}{2}\|w\|_2^2 - \lambda_1 \|w\|_1.$$

The lambda terms force the parameter vector to be small in the respective norm. This penalizes the model complexity and is known as **regularization**. The parameters lambda_2 and lambda_1 allow us to control the extent of regularization. The parameter vector `w^*` is then estimated as

$$w^* = \text{arg max}_{w\in \mathbb{R}^d} L(w)$$

If we estimated the vector `w^*`, we can find the most likely tag a sentence `s^*` for a sentence x by



$$s^* = \text{arg max}_{s} p(s|x; w^*).$$

### Implementation

#### Step 0: Install the needed modules
1.`sklearn_crfsuite`

Run `pip install sklearn_crfsuite` or 

`conda install -c derickl sklearn-crfsuite`

2.`eli5`

Run `pip install eli5` or 

`conda install -c conda-forge eli5`

#### Step I: Loading the data

Now we want to apply this model. Let’s start by loading the data.

In [1]:
import pandas as pd
import numpy as np

Make sure to download the data from Kaggle first from [this link](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus/downloads/entity-annotated-corpus.zip/4). Note that you will need to register and login in order to do that.

In [2]:
data = pd.read_csv("../../../data/NERC_datasets/entity-annotated-corpus/ner_dataset.csv", encoding="latin1")

In [3]:
data = data.fillna(method="ffill")

#### Step II: Initial analysis

Let's print the last 10 rows of the data:

In [4]:
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


As further analysis, we can make a set of all unique words:

In [5]:
words = list(set(data["Word"].values))

In [6]:
n_words = len(words); n_words

35178

So we have 47959 sentences containing 35178 unique words. 

We will use a class called SentenceGetter to retrieve sentences with their labels. Don't worry about the details of this.

In [8]:
pos = list(set(data["POS"].values))

In [14]:
print(pos)

['CC', 'NN', 'FW', 'UH', 'VBD', 'MD', 'JJS', 'RBR', 'RP', 'VB', 'JJR', 'NNPS', 'RRB', 'VBZ', 'WRB', 'PRP$', 'JJ', 'RB', 'DT', 'CD', 'VBN', 'TO', '.', ':', 'RBS', 'IN', 'NNS', 'PDT', ';', 'POS', 'EX', 'WDT', 'WP', 'VBG', '$', 'PRP', 'VBP', '``', 'NNP', ',', 'WP$', 'LRB']


In [15]:
labels = list(set(data["Tag"].values))

In [16]:
print(labels)

['B-per', 'I-gpe', 'I-geo', 'I-per', 'B-org', 'I-tim', 'B-nat', 'B-eve', 'B-gpe', 'I-nat', 'I-org', 'I-eve', 'O', 'B-art', 'B-geo', 'B-tim', 'I-art']


In [30]:
# Function that processes the data into sentences
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [18]:
getter = SentenceGetter(data)

In [19]:
sent = getter.get_next()

This is an example sentence we get with our SentenceGetter:

In [20]:
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


We can get all sentences as follows:

In [21]:
sentences = getter.sentences

In [23]:
print(len(sentences))

47959


In [24]:
sentence= sentences[3]
print(sentence)

[('They', 'PRP', 'O'), ('left', 'VBD', 'O'), ('after', 'IN', 'O'), ('a', 'DT', 'O'), ('tense', 'NN', 'O'), ('hour-long', 'JJ', 'O'), ('standoff', 'NN', 'O'), ('with', 'IN', 'O'), ('riot', 'NN', 'O'), ('police', 'NNS', 'O'), ('.', '.', 'O')]


#### Step III: Feature engineering

Now we craft a set of features and prepare the dataset.

In [31]:
# input is a sentence as a structure show above 
#and and ith word from the sentence to return the features for that word

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    # data structure consisting of a feature name and value for the token
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(), # lower case variant of the token
        'word[-3:]': word[-3:], #suffix of 3 characters
        'word[-2:]': word[-2:], #suffix of 2 characters
        'word.isupper()': word.isupper(), # initial captial
        'word.istitle()': word.istitle(), # all words ini caps
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2], #first two characters of the PoS Tag
    }
    if i > 0:
        # adding features for the word based on the previous word
        word1 = sent[i-1][0] # previous word
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        # adding features for the word based on the next word
        word1 = sent[i+1][0] # next word
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True # end of sentence

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

The following code extracts features with our functions above. It also prepares all labels from the original dataset.

In [32]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

#### Step IV: Initialize CRF

Now we can initialize the algorithm. We use the conditional random field (CRF) implementation provided by sklearn-crfsuite.

In [35]:
import sklearn_crfsuite

from sklearn_crfsuite import CRF

# different parameters are used for training
# check https://sklearn-crfsuite.readthedocs.io/en/latest/api.html?highlight=CRF
crf = CRF(algorithm='lbfgs',
          c1=0.1, #The coefficient for L1 regularization.
          c2=0.1, #The coefficient for L2 regularization.
          max_iterations=100,
          all_possible_transitions=False) #When True, CRFsuite generates transition features that associate all of possible label pairs, 
                                        #including ones that never occur. Suppose that the number of labels in the training data is L, this function will generate (L * L) transition features

5-fold cross-validation.

In [36]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

We will use the scikit-learn classification report to evaluate the tagger, because we are basically interested in precision, recall and the f1-score. These metrics are common in NLP tasks and if you are not familiar with these metrics, then check out the wikipedia articles.

#### Step V: Train and test the CRF algorithm

In [37]:
# given the model "crf", given the feature representations of the sentences X and their labels y,
# apply 5-folder cross classifcation, testing 5 times on 80% train and 20% test
# this may take half an hour depending on the machine you are running it
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [38]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

             precision    recall  f1-score   support

      B-art       0.37      0.11      0.17       402
      B-eve       0.52      0.35      0.42       308
      B-geo       0.85      0.90      0.88     37644
      B-gpe       0.97      0.94      0.95     15870
      B-nat       0.66      0.37      0.47       201
      B-org       0.78      0.72      0.75     20143
      B-per       0.84      0.81      0.82     16990
      B-tim       0.93      0.88      0.90     20333
      I-art       0.11      0.03      0.04       297
      I-eve       0.34      0.21      0.26       253
      I-geo       0.82      0.79      0.80      7414
      I-gpe       0.92      0.55      0.69       198
      I-nat       0.61      0.27      0.38        51
      I-org       0.81      0.79      0.80     16784
      I-per       0.84      0.89      0.87     17251
      I-tim       0.83      0.76      0.80      6528
          O       0.99      0.99      0.99    887908

avg / total       0.97      0.97      0.97  

This report shows that the peformance varies considerably across the different types of entities.
Also note that the class "O" has F1 of 97 and is the dominant class

In [39]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

#### Step VI: Inspect features

The nice thing about CRFs is, that we can look into the algorithm and visualize the transition probabilites from one tag to another. We also can see which features are important for predicting a certain tag. We use the eli5 library to perform the investigation: https://eli5.readthedocs.io/en/latest/

In [ ]:
import eli5

In [ ]:
eli5.show_weights(crf, top=30)

#### Step VII: Tuning the model

CRF just remembering a lot of words. For example for the tag ‘B-per’, the algorithm remembers ‘president’ ‘obama’. To overcome this issue we can tune the parameters, especially the regularization parameters of the CRF algorithm. The c1 and c2 parameter of the CRF algorithm are the regularization parameters \lambda_1 and \lambda_2. While c1 weights the l_1 regularization, the c2 parameter weights the l_2 regularization. We now limit the number of features used by enforcing sparsity on the parameter vector w. To do this we increase the l_1-regularization parameter c1.

In [ ]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [ ]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

In [ ]:
crf.fit(X, y)

Now we look again at the features.

In [ ]:
eli5.show_weights(crf, top=30)

As expected, we see, that the model stops to rely on words and uses the context more, as it generalizes better is more useful over multiple training instances. This is an effect of the l_1-regularization.

On regularization: "Regularization is a technique to discourage the complexity of the model. It does this by penalizing the loss function. This helps to solve the overfitting problem."

In particular, L1-regularization acts as a feature selector, simply removing some of the features. You can read more on regularization [here](https://medium.com/datadriveninvestor/l1-l2-regularization-7f1b4fe948f2).